In [66]:
# Import packages
import pandas as pd
import numpy as np
from typing import Any
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [75]:
def read_orders_table() -> Any:
    
    # Read the csv file
    orders_df = pd.read_csv("/Users/burakozen/Jupyter_Projects/data/olist_orders_dataset.csv")

    # Converting object types to datetime for further analysis
    orders_df.order_purchase_timestamp = pd.to_datetime(orders_df.order_purchase_timestamp)
    orders_df.order_approved_at = pd.to_datetime(orders_df.order_approved_at)
    orders_df.order_delivered_carrier_date = pd.to_datetime(orders_df.order_delivered_carrier_date)
    orders_df.order_delivered_customer_date = pd.to_datetime(orders_df.order_delivered_customer_date)
    orders_df.order_estimated_delivery_date = pd.to_datetime(orders_df.order_estimated_delivery_date)

    # Important Data Info 
    print("All order_ids are unique:",orders_df.customer_id.nunique()==len(orders_df))
    print("Max order purchase date:", orders_df.order_purchase_timestamp.max())
    print("Min order pruchase date:", orders_df.order_purchase_timestamp.min())
    print("Different order statuses:", orders_df.order_status.unique())
    
    return orders_df

In [76]:
def read_payments_table() -> Any:
    
    # Read the csv file
    payments_df = pd.read_csv("/Users/burakozen/Jupyter_Projects/data/olist_order_payments_dataset.csv")
    
    # Important Data Info
    print("Total number of rows:",len(payments_df))
    print("Total number of unique orders:",payments_df.order_id.nunique())
    print("Payment Types:", payments_df.payment_type.unique())
    print("Payment Value Mean:", payments_df.payment_value.mean())
    
    return payments_df

In [77]:
def read_customers_table() -> Any:
    
    # Read the csv file
    customers_df = pd.read_csv("/Users/burakozen/Jupyter_Projects/data/olist_customers_dataset.csv")
    
    # Important Data Info
    print("Total number of customers:", customers_df.customer_unique_id.nunique())
    print("Total number of different cities:", customers_df.customer_city.nunique())
    print("Total number of different states:", customers_df.customer_state.nunique())
    
    return customers_df

In [78]:
def extract_orders_general_features(orders_df) -> Any:
    
    # ORDER TABLE BASED FEATURE EXTRACTION - ORDER FEATURES

    # Drop rows which has NA values in the timestamp columns
    orders_df = orders_df[orders_df['order_approved_at'].notna() 
                          & orders_df['order_purchase_timestamp'].notna() 
                          & orders_df['order_delivered_carrier_date'].notna() 
                          & orders_df['order_approved_at'].notna()
                          & orders_df['order_estimated_delivery_date'].notna() 
                          & orders_df['order_delivered_customer_date'].notna()]


    # Data Sanity Check
    orders_df = orders_df.loc[~((orders_df['order_approved_at'] < orders_df['order_purchase_timestamp']) | (orders_df['order_delivered_carrier_date'] < orders_df['order_approved_at'])),:]

    # Computing new features: total_waiting & days_between_estimate_actual_delivery
    orders_df["payment_approvement_waiting"]=(orders_df.order_approved_at - orders_df.order_purchase_timestamp).dt.days
    orders_df["delivered_carrier_waiting"]=(orders_df.order_delivered_carrier_date - orders_df.order_approved_at).dt.days
    orders_df["total_waiting"] = orders_df.payment_approvement_waiting + orders_df.delivered_carrier_waiting
    orders_df["days_between_estimate_actual_delivery"]=(orders_df.order_estimated_delivery_date - orders_df.order_delivered_customer_date).dt.days

    # Select features to be returned
    orders_general_features=orders_df[['order_id','order_status','total_waiting','days_between_estimate_actual_delivery']]
    
    # Important Data Info
    print("All total_waiting values non-negative:", (orders_general_features.total_waiting>=0).all())
    print("Max of days between estimate and actual delivery:", orders_general_features.days_between_estimate_actual_delivery.max())
    print("Min of days between estimate and actual delivery:", orders_general_features.days_between_estimate_actual_delivery.min())
    
    return orders_general_features


In [79]:
def extract_orders_payments_features(payments_df) -> Any:
    
    # PAYMENT TABLE BASED FEATURE EXTRACTION - ORDER FEATURES

    # Drop rows which has NA values in the timestamp columns
    payments_df = payments_df[payments_df['payment_type'].notna() & payments_df['payment_value'].notna()] 


    # Computing new features: use_voucher & total_payment & payment_type
    payments_df_agg=payments_df\
    .assign(is_voucher= np.where(payments_df['payment_type']=='voucher',1,0))\
    .groupby(['order_id'],as_index=False) \
    .agg(use_voucher=("is_voucher","max"), \
         total_payment=("payment_value","sum"), \
         payment_type=("payment_type","max") \
        )

    # Select columns to be returned
    orders_payments_features=payments_df_agg[['order_id','use_voucher','total_payment','payment_type']]

    # Important Data Info
    print("All total_payment values non-negative:", (orders_payments_features.total_payment>=0).all())
    
    return orders_payments_features

In [80]:
def extract_customer_features(customers_df) -> Any:
    
    # CUSTOMER TABLE BASED FEATURE EXTRACTION - CUSTOMER FEATURES

    # Merge 2 dataframes
    orders_customers_merged = orders_df.merge(customers_df,left_on='customer_id',right_on='customer_id',how='left')

    # Compute a new feature: multiple_order
    orders_customers_merged["total_orders"]=orders_customers_merged.groupby('customer_unique_id')['customer_id'].transform('count')
    orders_customers_merged['multiple_order'] = np.where(orders_customers_merged['total_orders']> 1, 1, 0)

    #Compute a new feature: days_since_order
    dataset_max_date=orders_customers_merged.order_purchase_timestamp.max()
    orders_customers_merged['days_since_order'] = (dataset_max_date-orders_customers_merged.order_purchase_timestamp).dt.days

    # Filter out only the first orders of users in the dataset
    orders_customers_merged["order_rank"]=orders_customers_merged.groupby('customer_unique_id')['order_purchase_timestamp'].rank(method='first')
    orders_customers_merged=orders_customers_merged[orders_customers_merged["order_rank"]==1.0].drop(columns=['order_rank'])

    # Select columns to be returned and rename them accordingly
    orders_customers_merged = orders_customers_merged.rename(columns={"order_id": "first_order_id", "days_since_order": "days_since_first_order"})
    customer_features = orders_customers_merged[['customer_unique_id','customer_city','customer_state','first_order_id','days_since_first_order','multiple_order']]

    # Important Data Info
    print("Ratio of multiple-orders customers over single-order customers:",round(customer_features.multiple_order.value_counts()[1]/customer_features.multiple_order.value_counts()[0],2))

    return customer_features


In [81]:
def generate_training_data(orders_general_features, orders_payments_features, customer_features) -> Any:

    # TRAINING DATA GENERATION

    # Merge dataframes and drop irrelevant columns 
    order_features = orders_general_features.merge(orders_payments_features,left_on='order_id',right_on='order_id',how='left')
    training_data_raw = customer_features.merge(order_features,left_on='first_order_id',right_on='order_id',how='left').drop(columns=['order_id','order_status'])

    # Rename columns
    training_data_raw = training_data_raw.rename(columns={
                                      "total_waiting": "first_order_total_waiting", 
                                      "days_between_estimate_actual_delivery": "first_order_days_between_estimate_actual_delivery",
                                      "use_voucher": "first_order_use_voucher",
                                      "total_payment": "first_order_total_payment",
                                      "payment_type": "first_order_payment_type"
                                     })

    # Decrease number of dimensions in the customer_city and customer_state columns to 6 (before applyling one-hot-encoding)
    top5_cities = ["sao paulo","rio de janeiro","belo horizonte","brasilia","curitiba"]
    top5_states = ["SP","RJ","MG","RS","PR"]
    training_data_raw['customer_city'] = training_data_raw['customer_city'].apply(lambda city: city if city in top5_cities else 'other')
    training_data_raw['customer_state'] = training_data_raw['customer_state'].apply(lambda state: state if state in top5_states else 'other')

    # Create a label column 'churned': If multiple_order is 1, then CHURNED=0, otherwise CHURNED=1. Select only the churned customers if it has been more than 365 days since the first order
    training_data_raw['churned'] = 1 
    training_data_raw.loc[training_data_raw.multiple_order == 1, 'churned'] = 0
    training_data_raw = training_data_raw.loc[ (training_data_raw.churned==0) | ((training_data_raw.churned==1) & (training_data_raw.days_since_first_order > 365))]

    #Select columns to be returned and drop NA rows
    training_data = training_data_raw.drop(columns=['multiple_order']).dropna()
    
    # Important Data Info
    print("Number of training data records:", len(training_data))
    print("Churn user ratio after first order:", round(training_data.churned.value_counts()[1]/len(training_data),2))

    return training_data


In [83]:
def train_churn_model(training_data) -> Any:
    
    # Parameters for data split
    test_size_fraction = 0.33
    random_seed = 42

    # Data Split
    X_train, X_test, Y_train, Y_test = train_test_split(training_data.drop(columns=['customer_unique_id','first_order_id','churned']),
                                                        training_data.churned,
                                                        test_size=test_size_fraction,
                                                        random_state=random_seed)

    # Define a One-Hot Encoder Transformer
    categorical_cols = ['customer_city','customer_state','first_order_payment_type']
    transformer = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)],remainder='passthrough')
    
    
    # Model: Define a Gradient Boosting Classifier
    learning_rate = 0.01
    max_depth = 6
    max_features = 'sqrt'
    min_samples_leaf = 10
    n_estimators = 100
    subsample = 0.8
    random_state = 42
    
    model = GradientBoostingClassifier(learning_rate=learning_rate,
                                       max_depth=max_depth,
                                       max_features=max_features,
                                       min_samples_leaf=min_samples_leaf,
                                       n_estimators=n_estimators,
                                       subsample=subsample,
                                       random_state=random_state)

    # Pipeline Fit
    pipeline = Pipeline(steps=[('t', transformer), ('m', model)])
    pipeline.fit(X_train, Y_train)

    # Model Evaluation
    # 1. Predict probabilities of target 1:Churned
    probs = pipeline.predict_proba(X_test)[:,1]
    # 2. Calculate average precision and area under the receiver operating characteric curve (ROC AUC)
    avg_precision = average_precision_score(Y_test, probs, pos_label=1)
    auc = roc_auc_score(Y_test, probs)

    # Important Model Metrics
    print("Average Precision Value:",avg_precision)
    print("Area under ROC:",auc)
    
    return pipeline

In [86]:
print("\n--- read_orders_table starts ---")
orders_df = read_orders_table()
print("--- read_orders_table ends ---")

print("\n--- read_payments_table starts ---")
payments_df = read_payments_table()
print("--- read_payments_table ends ---")

print("\n--- read_customers_table starts ---")
customers_df = read_customers_table()
print("--- read_customers_table ends ---")

print("\n--- extract_orders_general_features starts ---")
orders_general_features  = extract_orders_general_features(orders_df)
print("--- extract_orders_general_features ends---")

print("\n--- extract_orders_payments_features starts ---")
orders_payments_features = extract_orders_payments_features(payments_df)
print("--- extract_orders_payments_features ends ---")

print("\n--- extract_customer_features starts ---")
customer_features = extract_customer_features(customers_df)
print("--- extract_customer_features ends ---")

print("\n--- generate_training_data starts ---")
training_data = generate_training_data(orders_general_features,orders_payments_features,customer_features)
print("--- generate_training_data ends ---")

print("\n--- train_churn_model starts ---")
churn_model = train_churn_model(training_data)
print("--- train_churn_model ends ---")


--- read_orders_table starts ---
All order_ids are unique: True
Max order purchase date: 2018-10-17 17:30:18
Min order pruchase date: 2016-09-04 21:15:19
Different order statuses: ['delivered' 'invoiced' 'shipped' 'processing' 'unavailable' 'canceled'
 'created' 'approved']
--- read_orders_table ends ---

--- read_payments_table starts ---
Total number of rows: 103886
Total number of unique orders: 99440
Payment Types: ['credit_card' 'boleto' 'voucher' 'debit_card' 'not_defined']
Payment Value Mean: 154.10038041699553
--- read_payments_table ends ---

--- read_customers_table starts ---
Total number of customers: 96096
Total number of different cities: 4119
Total number of different states: 27
--- read_customers_table ends ---

--- extract_orders_general_features starts ---
All total_waiting values non-negative: True
Max of days between estimate and actual delivery: 146
Min of days between estimate and actual delivery: -189
--- extract_orders_general_features ends---

--- extract_orde